In [1]:
import torch
#import gradio as gr

from textwrap import fill
from IPython.display import Markdown, display

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, AutoConfig

import warnings
warnings.filterwarnings('ignore')
print(torch.cuda.is_available())
NUM_CHAPTERS = 10
MAIN_NAME = "Klaus"

True


In [2]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
#MODEL_NAME = "NousResearch/Yarn-Mistral-7b-128k"
#MODEL_NAME = 'mosaicml/mpt-7b-storywriter'

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    #use_flash_attention = True,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin C:\Users\phili\miniconda3\envs\ChildrensBook\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121_nocublaslt.dll
CUDA SETUP: CUDA runtime path found: C:\Users\phili\miniconda3\envs\ChildrensBook\bin\cudart64_12.dll
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary C:\Users\phili\miniconda3\envs\ChildrensBook\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121_nocublaslt.dll...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# name = 'mosaicml/mpt-7b-storywriter'
# 
# config = AutoConfig.from_pretrained(name, trust_remote_code=True)
# # config.attn_config['attn_impl'] = 'triton'
# config.init_device = 'cuda:0' 
# config.max_seq_len = 83968 # (input + output) tokens can now be up to 83968
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
# )
# 
# model = AutoModelForCausalLM.from_pretrained(
#   name,
#   config=config,
#   torch_dtype=torch.bfloat16, # Load model weights in bfloat16
#   trust_remote_code=True,
#   device_map="auto",
#   quantization_config=quantization_config
# )
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")


In [4]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 2000
generation_config.min_new_tokens = -1
#generation_config.max_length = 10000
generation_config.top_p = 0.9
generation_config.top_k = 50
#generation_config.do_sample = True
generation_config.temperature = 0.7
#generation_config.repetition_penalty = 1

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)
pipeline.model.config.pad_token_id = pipeline.model.config.eos_token_id

In [5]:
llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

In [6]:
longchat_template = f"""
[INST] Objective: Create a {NUM_CHAPTERS}-chapter children's book set in a magical Korean world with dragons. Each 1000-word chapter features a unique, open-ended adventure.
AI's Role:
Craft engaging, informative stories for young readers.
Use a friendly tone, rich descriptions, diverse characters.
Focus on positive themes like friendship and learning.
Mix in humor and educational elements.
Build resilience and empathy through challenges.
End stories with hope or lessons, avoiding negativity.
Process: Write one detailed chapter at a time, awaiting user prompts for subsequent chapters.

Prologue: The Quest for Harmony in Korea's Enchanted Realms

In a realm where ancient mystique and modern marvels intertwine, an epic saga awaits to be told. Here, in the heart of Korea's enchanted lands, a world brimming with magic and mystery beckons. A world where dragons trace majestic arcs across the sky and mythical beings whisper the secrets of the ages.

In this realm of wonder, two heroes are predestined to unite: {MAIN_NAME}, the resplendent phoenix from the soaring cliffs of Pohang, a creature of fire and lore, and Nubzuki, the inquisitive platfish from the tranquil waters of Daejeon, wise and whimsical.

Their journey, a tapestry of peril and enlightenment, will sweep them through realms where ancient trees murmur forgotten tales, across vast meadows echoing with celestial music, and into vibrant cities where history and future meld in perfect harmony.

This odyssey, however, is far from a mere escapade; it's a quest of discovery, resilience, and kinship. Through trials and tribulations, {MAIN_NAME} and Nubzuki will delve deep into the heart of Korean culture, encountering its rich traditions, arts, and ancient wisdoms, each step a lesson in unity and strength.

But lurking in the shadows is a darkness, a force that seeks to unbalance this world of wonder. Our heroes, {MAIN_NAME} and Nubzuki, must stand as one against this burgeoning chaos. In their quest, they will learn that true might blossoms from togetherness and that courage can be found in the most unexpected places.

So, brace yourselves, dear readers, for an adventure beyond compare. Soar with {MAIN_NAME} into realms uncharted, dive with Nubzuki into the depths of the extraordinary. Together, we will traverse a world where Korean lore comes alive, where each twist and turn is a doorway to the extraordinary.

Prepare to embark on "The Quest for Harmony in Korea's Enchanted Realms," a narrative pulsating with magic, brimming with lessons, and shimmering with the light of enduring hope.
"""
longchat_template += """
Current conversation:
{history}
USER: {input}
AI: [/INST]
"""

In [7]:
longchat_prompt_template = PromptTemplate(
    input_variables=["input", "history"], template=longchat_template
)

In [8]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(ai_prefix="AI", human_prefix="USER"),
    prompt=longchat_prompt_template,
)

In [9]:
# from langchain.chains.conversation.memory import ConversationSummaryMemory
# 
# conversation_buf = ConversationChain(
# 	llm=llm,
# 	memory=ConversationSummaryMemory(llm=llm, ai_prefix="AI", human_prefix="USER"),
#   prompt=longchat_prompt_template
# )

In [10]:
# from langchain.memory import ConversationSummaryBufferMemory
# 
# conversation_buf = ConversationChain(
#     llm=llm, 
#     prompt=longchat_prompt_template,
#     memory=ConversationSummaryBufferMemory(
#         llm=llm,
#         max_token_limit=1300,
#         ai_prefix="AI", 
#         human_prefix="USER")
# )

In [11]:
# from langchain.memory import ConversationKGMemory
# 
# conversation_buf = ConversationChain(
# 	llm=llm,
# 	memory=ConversationKGMemory(llm=llm),
#   prompt=longchat_prompt_template
# )

In [12]:
print(conversation_buf("Write now only the first chapter!")["response"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
print(conversation_buf(f"[INST] Create now 4 decisions how the story could continue. [/INST]")["response"])

In [ ]:
import random
random.seed()
num_dec = 4  # number of decisions
for i in range(NUM_CHAPTERS - 2):
    decision = random.randint(1, num_dec)
    print(f"Decision: {decision}")
    print(conversation_buf(f" Write now the next chapter with using decision {decision}. [/INST]")["response"])
    print(conversation_buf(f"[INST] Create now 4 decisions how the story could continue. [/INST]")["response"])

In [ ]:
decision = "c"
print(conversation_buf(f"[INST] Write now the next chapter with using decision {decision}. Find a happy or sad ending based on the previous decisions. [/INST]")["response"])

In [ ]:
print(conversation_buf(f"[INST] Write now a detailed text to evaluate the decisions of the Reader, and show what was good or bad and why! Please call the reader by you. [/INST]")["response"])

In [ ]:
print(conversation_buf.memory.buffer)